In [3]:
import tkinter as tk
from tkinter import filedialog
import requests
import json
import pandas as pd
from PIL import Image, ImageTk
import os

# API endpoint URLs
VALIDATE_API_KEY_URL = "https://validator.extracttable.com/"
TRIGGER_REQUEST_URL = "https://trigger.extracttable.com/"
GET_RESULT_URL = "https://getresult.extracttable.com/"

# Your API key
API_KEY = "evyJiffxeU2mVoiAmghKTLa99BS4V81a7k56OhNp"

# Function to validate API key
def validate_api_key():
    headers = {"x-api-key": API_KEY}
    response = requests.get(VALIDATE_API_KEY_URL, headers=headers)
    if response.status_code == 200:
        print("API Key is valid")
    else:
        print("Invalid API Key")

# Function to trigger a job to process the file
def trigger_request(file_data):
    headers = {"x-api-key": API_KEY}
    files = {"input": file_data}
    response = requests.post(TRIGGER_REQUEST_URL, files=files, headers=headers)
    if response.status_code == 200:
        data = response.json()
        job_id = data.get("JobId")
        print(f"Job triggered successfully. JobId: {job_id}")
        return job_id
    else:
        print("Failed to trigger job")
        return None

# Function to retrieve the result of the triggered job
def get_result(job_id, file_path):
    url = f"{GET_RESULT_URL}?JobId={job_id}"
    headers = {"x-api-key": API_KEY}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        data = response.json()
        job_status = data.get("JobStatus")
        if job_status == "Success":
            tables = data.get("Tables")
            print("Tabular data extracted successfully:")
            for i, table in enumerate(tables):
                df = pd.DataFrame(table)
                # Write DataFrame to Excel file
                attendance_file = f"{os.path.splitext(file_path)[0]}_attendance.xlsx"
                df.to_excel(attendance_file, index=False)
                print(f"Attendance sheet saved to {attendance_file}")
                return attendance_file
        else:
            print(f"Job failed with status: {job_status}")
    else:
        print("Failed to retrieve result")

# Function to handle image drop event
def on_drop(event):
    # Get file path
    file_path = event.data

    # Check if the dropped file is an image
    if file_path.lower().endswith(('.png', '.jpg', '.jpeg')):
        # Display dropped image
        load_image(file_path)
        # Trigger attendance processing
        process_attendance(file_path)
    else:
        print("Please drop an image file.")

# Function to load and display dropped image
def load_image(file_path):
    img = Image.open(file_path)
    img = img.resize((300, 300), Image.ANTIALIAS)
    photo = ImageTk.PhotoImage(img)
    image_label.config(image=photo)
    image_label.image = photo

# Function to process attendance
def process_attendance(file_path):
    # Trigger request to process the file
    file_data = open(file_path, "rb")
    job_id = trigger_request(file_data)
    if job_id:
        # Retrieve the result of the triggered job
        attendance_file = get_result(job_id, file_path)
        if attendance_file:
            # Enable download button
            download_button.config(state=tk.NORMAL)

# Function to handle download button click event
def download_attendance():
    # Open file dialog to save attendance sheet
    file_path = filedialog.asksaveasfilename(defaultextension=".xlsx", filetypes=[("Excel files", "*.xlsx")])
    if file_path:
        # Move generated attendance sheet to selected location
        os.replace(attendance_file, file_path)
        print(f"Attendance sheet saved to {file_path}")

# Create main window
root = tk.Tk()
root.title("Attendance Marking")

# Validate API key
validate_api_key()

# Create drop target label
drop_label = tk.Label(root, text="Drop image here to process attendance", bg="lightgray", font=("Arial", 12), padx=10, pady=10)
drop_label.pack(fill=tk.BOTH, expand=True)

# Bind drop event to drop label
drop_label.bind("<Drop>", on_drop)
drop_label.bind("<DragEnter>", lambda event: drop_label.config(bg="lightblue"))
drop_label.bind("<DragLeave>", lambda event: drop_label.config(bg="lightgray"))

# Create image label
image_label = tk.Label(root)
image_label.pack()

# Create download button (disabled by default)
download_button = tk.Button(root, text="Download Attendance Sheet", command=download_attendance, state=tk.DISABLED)
download_button.pack(pady=10)

# Run the application
root.mainloop()


ModuleNotFoundError: No module named 'pandas'

SyntaxError: invalid syntax (1139968535.py, line 1)